# Jupyter Notebook Delft FEWS User Day 2019

### by Lindsay Millard, Hydrologist Seqwater



## Overview

Jupyter Notebooks are fast way to develop and trial code for Delft FEWS adapters
visit:
https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks
to learn more about Jupyter Notebooks
Bring to your particular attention the 
####  Pandas for Data Analysis 
Example notebooks - such as Timeseries Analysis


Let start off by calling the following Python modules to run the code in this example. Most are standard libraries within a Python installation. <br> If your Anaconda Distribution (or vanilla Python) does not have it then you can use either conda or pip to install them.

Command Line >>> Pip install 'missing module'


In [106]:
from xml.etree.ElementTree import *  # Import everything 

import time
from datetime import datetime
import csv
import os

## from xlsxwriter.workbook import Workbook

import numpy as np
import pandas as pd  ## Abbreviate it using the convention

# Check what version of the module you are running, lets check Pandas.
pd.__version__

'0.23.0'

You can set up functions for checking the currency of the file found in the system path.  
This needs to be run before the main part of the code

In [107]:
## Worker Function for checking currency
def newest(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

# It is a good idea to put your variables up front
fewsNamespace="http://www.wldelft.nl/fews/PI"

# The working folder root path can be defined here and then built on later when looking for files or output paths
regionHome = r'W:\500_Reference\530_Presentations\04_FEWS User Conference - Australia\2019\Python_LinguaFrancaofFEWS\GoldSim/'

regionHome = r'C:\Users\lmillard\GitRepos\DFUDA_2019_Adapter/'

Using For and If loops are a great way to pull data into the Python interpreter. It is probably a little confusing at first but the commentary along the way should make aspects a little clearer. If it doesn't make sense it can be unwrapped into a simple example. 

In [108]:
# This is a list of the names of the XML files to go look for. I.e. look in regionHome for import.xml
XMLs = ['importState','import']

for xml in XMLs:
    fin = os.path.join(regionHome,'1_Input',xml+'.xml')    
    FEWS_Export = fin   #  regionHome+'1_Input/importState.xml'

    parameterNames = ['HNPD_OUT','Reservoir.inflow.forecast', 'Reservoir.outflow.forecast', 'Gate.setting.forecast',]
    spreadsheetNames = {'Reservoir.inflow.forecast':'LakeInflows', 'Reservoir.outflow.forecast':'RegulatorFlows', 'Gate.setting.forecast':'Gates', 'HNPD_OUT':'LakeLevels'}

XMLs = ['importState']#,'import']
## This will only work one by one in a Jupyter Notebook with many cells spilting the loop

for xml in XMLs:
    print(xml)
    fin = os.path.join(regionHome,'1_Input',xml+'.xml')    
    FEWS_Export = fin   #  regionHome+'1_Input/importState.xml'

    parameterNames = ['HNPD_OUT','Reservoir.inflow.forecast', 'Reservoir.outflow.forecast', 'Gate.setting.forecast',]
    spreadsheetNames = {'Reservoir.inflow.forecast':'LakeInflows', 'Reservoir.outflow.forecast':'RegulatorFlows', 'Gate.setting.forecast':'Gates', 'HNPD_OUT':'LakeLevels'}

    for parametername in parameterNames:                   
        ParList=[]
        par =[]
        locId=[]
        i = 0    
        

importState


Here is reminder of what an XML looks like (using Grid display in XML spy)

<img src="assets/XML_timeseries.png">

Here is some code the will scrape timeseries data out of an XML file exported from FEWS.  
The header information can be dealt with first. You can retrieve the number of Series (7), ParameterId (Reservoir.inflow.forecast) and MissingValue (-999.0)

In [109]:
        with open(FEWS_Export, "r") as file:
            print('We have opended the following file:\n'+FEWS_Export)
            
            tree = parse(file)
            PItimeSeries = tree.getroot()
            
            series = PItimeSeries.findall('.//{' + fewsNamespace + '}series')
            ## Search for all the Parameters defined in the XML
            Parameters=PItimeSeries.findall('.//{' + fewsNamespace + '}parameterId')
            for param in Parameters:
                ParList.append(param.text)
            
            parCount=0  # Initialise a counter of parameters found                
            
            missingvalue = PItimeSeries.findall('.//{' + fewsNamespace + '}missVal')
            

We have opended the following file:
C:\Users\lmillard\GitRepos\DFUDA_2019_Adapter/1_Input\importState.xml


The code does not know how many timeseries are in the XML. The following code will loop through all of the parameters of interest and then count the length.  

In [110]:
            for S in series:
                events = S.findall('.//{' + fewsNamespace + '}event')
                
                # When working with Date Time we will need to define it is a DateTime not a string or float
                ArrayDates = np.zeros((len(events)),dtype='datetime64[ns]')
                
                par = S.find('.//{' + fewsNamespace + '}parameterId').text
                if par==parametername:
                    parCount = parCount + 1
            
            ## Once you loop through the file you are able to set up empty array of appropriate size            
            ArrayValues = np.zeros((len(events), parCount))
            print('Number of Timestep rows:     '+ str(ArrayValues.shape[0]))
            print('Number of Parameter columns: '+ str(ArrayValues.shape[1]))
            

Number of Timestep rows:     481
Number of Parameter columns: 0


In [111]:
            j = 0

            DateList = []
            for S in series:
                par = S.find('.//{' + fewsNamespace + '}parameterId').text
                
                if par!=parametername:
                    pass
                
                else:            
                    events = S.findall('.//{' + fewsNamespace + '}event')
                    locs = S.findall('.//{' + fewsNamespace + '}locationId')
                    
                    i=0
                    for l in locs:
                        locId.append(l.text)        
                
                    for ev in events:
                        if ev.attrib['value'] == S.find('.//{' + fewsNamespace + '}missVal').text:
                            ArrayValues[i,j] = float(0)
                        else:
                            ArrayValues[i,j] = float(ev.attrib['value'])
                        
                        strucTime_Tuple = datetime.strptime(ev.attrib['date'] + " " + ev.attrib['time'],"%Y-%m-%d %H:%M:%S")
                        ArrayDates[i] = strucTime_Tuple

                        i += 1
                    j += 1
            row = i
            col = j
            print(row,col)

            DFd = pd.DataFrame(ArrayDates) 
            # Rename the columns
            DFd.columns = ['DateTime']
            
            DFv = pd.DataFrame(ArrayValues) 
            # Rename the columns
            DFv.columns = locId
            
            # Concatenate both columns together, do this column-wise (axis=1)
            DF = pd.concat([DFd,DFv],axis=1)

0 0


In [112]:
        ## Because the list of files and parameters contradict, we need to stop it overwriting the previous loop
        if xml == 'importState' and parametername == 'HNPD_OUT':
            GoldSIMSpreadSheet = regionHome+'2_Model/'+spreadsheetNames[parametername]+'.xlsx'

            #Let us use Pandas to write the file to a spreadsheet, this can be done in one line
            DF.to_excel(GoldSIMSpreadSheet,sheet_name='FEWS_Export')
            print(parametername)

        elif parametername != 'HNPD_OUT':
            GoldSIMSpreadSheet = regionHome+'2_Model/'+spreadsheetNames[parametername]+'.xlsx'
            DF.to_excel(GoldSIMSpreadSheet,sheet_name='FEWS_Export')
            print(parametername)

print(parametername)
DF.head()           

Gate.setting.forecast
Gate.setting.forecast


,DateTime
0,1970-01-01
1,1970-01-01
2,1970-01-01
3,1970-01-01
4,1970-01-01


# Part 1 - Run the Pre- Adapter

## Run the Master For loop from the top

This will allow the file to iterate over all of the parameters and files of interest.  
This will produce the XLSX files that GoldSim requires to run


In [115]:
XMLs = ['importState','import']

for xml in XMLs:
    fin = os.path.join(regionHome,'1_Input',xml+'.xml')    
    FEWS_Export = fin   #  regionHome+'1_Input/importState.xml'

    parameterNames = ['HNPD_OUT','Reservoir.inflow.forecast', 'Reservoir.outflow.forecast', 'Gate.setting.forecast',]
    spreadsheetNames = {'Reservoir.inflow.forecast':'LakeInflows', 'Reservoir.outflow.forecast':'RegulatorFlows', 'Gate.setting.forecast':'Gates', 'HNPD_OUT':'LakeLevels'}

    for parametername in parameterNames:                   
        ParList=[]
        par =[]
        locId=[]
        i = 0    
        
        with open(FEWS_Export, "r") as file:
            tree = parse(file)
            PItimeSeries = tree.getroot()
            Parameters=PItimeSeries.findall('.//{' + fewsNamespace + '}parameterId')
            for param in Parameters:
                ParList.append(param.text)
                
            missingvalue = PItimeSeries.findall('.//{' + fewsNamespace + '}missVal')
            series = PItimeSeries.findall('.//{' + fewsNamespace + '}series')

            # determine size of aray needed. all series MUST be the same length
            parCount=0
            for S in series:
                events = S.findall('.//{' + fewsNamespace + '}event')
                ArrayDates = np.zeros((len(events)),dtype='datetime64[ns]')
                
                par = S.find('.//{' + fewsNamespace + '}parameterId').text
                if par==parametername:
                    parCount = parCount + 1
            
            ArrayValues = np.zeros((len(events), parCount))
            j = 0

            DateList = []
            for S in series:
                par = S.find('.//{' + fewsNamespace + '}parameterId').text
                
                if par!=parametername:
                    pass
                
                else:            
                    events = S.findall('.//{' + fewsNamespace + '}event')
                    locs = S.findall('.//{' + fewsNamespace + '}locationId')
                    i=0
                    for l in locs:
                        locId.append(l.text)        
                
                    for ev in events:
                        if ev.attrib['value'] == S.find('.//{' + fewsNamespace + '}missVal').text:
                            ArrayValues[i,j] = float(0)
                        else:
                            ArrayValues[i,j] = float(ev.attrib['value'])
                        
                        strucTime_Tuple = datetime.strptime(ev.attrib['date'] + " " + ev.attrib['time'],"%Y-%m-%d %H:%M:%S")
                        ArrayDates[i] = strucTime_Tuple

                        i += 1
                    j +=  1
            row = i
            col = j

            DFd = pd.DataFrame(ArrayDates) 
            DFd.columns = ['DateTime']
            DFv = pd.DataFrame(ArrayValues) 
            DFv.columns = locId

            DF = pd.concat([DFd,DFv],axis=1)
                
            
        if xml == 'importState' and parametername == 'HNPD_OUT':
            GoldSIMSpreadSheet = regionHome+'2_Model/'+spreadsheetNames[parametername]+'.xlsx'
            DF.to_excel(GoldSIMSpreadSheet,sheet_name='FEWS_Export')
        
        elif parametername != 'HNPD_OUT':
            GoldSIMSpreadSheet = regionHome+'2_Model/'+spreadsheetNames[parametername]+'.xlsx'
            DF.to_excel(GoldSIMSpreadSheet,sheet_name='FEWS_Export')




# Part 2 - Run GoldSim

The Excel spreadsheet files for the:
    Gates, LakeInflows, LakeLevels and RegulatorFlows have all been produced by Part 1.
    
Running the GoldSim Player file produces RoutedFlows.txt and RoutedLevels.txt

<img src="assets/GoldSim_RunTheTimeseries.png">

# Part 3 - Run the Post-Adapter

Using the RoutedFlows.txt and RoutedLevel.txt we can now turn them back into XML timeseries for FEWS to read